In [1]:
import snowflake.connector
import pandas as pd

from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

CONEXÃO

In [2]:
from dotenv import load_dotenv
import os

load_dotenv('config.env')

user = os.getenv('user')
password = os.getenv('password')
account = os.getenv('account')
warehouse = os.getenv('warehouse')
database = os.getenv('database')
schema = os.getenv('schema')

In [3]:
conn = snowflake.connector.connect(
    user=user
    , password=password
    , account=account 
    , warehouse=warehouse
    , database=database   
    , schema=schema       
)

In [3]:
cursor = conn.cursor()

In [ ]:
cursor.close()
conn.close()

CARREGAR OS DADOS

In [4]:
query = """SELECT DISTINCT
            NM_CLIENTE
            , REGIAO
        from agg_sales_store """

cursor.execute(query)
rows = cursor.fetchall()

columns = [desc[0] for desc in cursor.description]
df_regiao = pd.DataFrame(rows, columns=columns)

print(df_regiao.head(100))

                         NM_CLIENTE      REGIAO
0                 Extreme Toy Store     Alabama
1    Games and Sport Supply Company     Alabama
2             Pedal Systems Company     Alabama
3                Racing Association     Alabama
4                 First Cycle Store     Arizona
..                              ...         ...
95                  Small Bike Shop  California
96            Sports Products Store  California
97    Technical Parts Manufacturing  California
98  The Bicycle Accessories Company  California
99               The New Bike Store  California

[100 rows x 2 columns]


In [ ]:
df_cliente['DATA_REF'] = pd.to_datetime(df_cliente['DATA_REF'])
df_cliente['MES'] = df_cliente['DATA_REF'].dt.month
df_cliente['ANO'] = df_cliente['DATA_REF'].dt.year
df_cliente['PMES'] = df_cliente['DATA_REF'].dt.to_period('M')

In [5]:
df_previsao = pd.read_csv('8-previsao_demanda.csv')
print(df_previsao)

      Unnamed: 0            NOME_PRODUTO                      NM_CLIENTE  \
0              0  Mountain-200 Black, 46        Professional Cycle Store   
1              1  Mountain-200 Black, 46             Real Sporting Goods   
2              2  Mountain-200 Black, 46                Many Bikes Store   
3              3  Mountain-200 Black, 46                 Resale Services   
4              4  Mountain-200 Black, 46                 Small Bike Shop   
...          ...                     ...                             ...   
1726        1726        Road-650 Red, 44        Advanced Bike Components   
1727        1727        Road-650 Red, 44               Fitness Toy Store   
1728        1728        Road-650 Red, 44         Totes & Baskets Company   
1729        1729        Road-650 Red, 44  Sure & Reliable Sporting Goods   
1730        1730        Road-650 Red, 44                  Good Bike Shop   

      forecast_1  forecast_2  forecast_3  
0            2.5         2.8         3.0  
1

In [6]:
df_joined = pd.merge(df_previsao, df_regiao, on='NM_CLIENTE', how='inner') 
print(df_joined)

      Unnamed: 0            NOME_PRODUTO                      NM_CLIENTE  \
0              0  Mountain-200 Black, 46        Professional Cycle Store   
1              1  Mountain-200 Black, 46             Real Sporting Goods   
2              2  Mountain-200 Black, 46                Many Bikes Store   
3              3  Mountain-200 Black, 46                 Resale Services   
4              4  Mountain-200 Black, 46                 Small Bike Shop   
...          ...                     ...                             ...   
1743        1726        Road-650 Red, 44        Advanced Bike Components   
1744        1727        Road-650 Red, 44               Fitness Toy Store   
1745        1728        Road-650 Red, 44         Totes & Baskets Company   
1746        1729        Road-650 Red, 44  Sure & Reliable Sporting Goods   
1747        1730        Road-650 Red, 44                  Good Bike Shop   

      forecast_1  forecast_2  forecast_3      REGIAO  
0            2.5         2.8    

In [7]:
df_grouped = df_joined.groupby('REGIAO')[['forecast_1', 'forecast_2', 'forecast_3']].sum()


df_grouped_sorted = df_grouped.sort_values(by='forecast_1', ascending=False)


print(df_grouped_sorted)

                forecast_1  forecast_2  forecast_3
REGIAO                                            
Canada              1593.3      1564.2      1549.4
California           727.0       725.0       723.9
Texas                706.9       703.9       700.9
Washington           454.2       451.2       448.7
France               396.1       392.7       390.9
United Kingdom       316.4       311.3       308.9
Colorado             296.7       291.5       288.3
Michigan             273.4       268.5       265.1
New York             191.0       191.9       193.8
New Hampshire        186.7       184.9       184.5
Missouri             149.5       147.6       146.6
Florida              149.5       149.2       150.6
Arizona              126.8       122.6       119.9
Tennessee            124.7       122.2       120.4
New Mexico           122.2       119.9       119.4
Connecticut           99.5       101.5       102.7
Virginia              88.0        83.8        81.8
Nevada                85.8     

In [8]:
df_grouped_sorted.to_csv('10-demanda_regiao.csv')